## Wrapper WatsonX v1

In [1]:
import logging
from typing import Any, Dict, List, Optional, Sequence, Type, Union, Callable, Literal
from pydantic import BaseModel, Field
from langchain_core.tools import BaseTool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.runnables import Runnable
from langchain_ibm import WatsonxLLM as BaseWatsonxLLM
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_core.outputs import LLMResult, Generation, GenerationChunk
from langchain_core.messages import BaseMessage
from langchain_core.language_models import LanguageModelInput

logger = logging.getLogger(__name__)

class WatsonxLLM(BaseWatsonxLLM):
    """Extended IBM watsonx.ai large language models."""

    bound_tools: Optional[List[BaseTool]] = Field(default=None, exclude=True)

    def __init__(self, *args, tools: Optional[List[BaseTool]] = None, **kwargs):
        super().__init__(*args, **kwargs)
        self.bound_tools = tools or []

    def _generate(
        self,
        prompts: List[str],
        stop: Optional[List[str]] = None,
        run_manager: Optional[Any] = None,
        stream: Optional[bool] = None,
        **kwargs: Any,
    ) -> LLMResult:
        """Call the IBM watsonx.ai inference endpoint which then generate the response.
        Args:
            prompts: List of strings (prompts) to pass into the model.
            stop: Optional list of stop words to use when generating.
            run_manager: Optional callback manager.
            stream: Optional boolean flag to indicate streaming generation.
        Returns:
            The full LLMResult output.
        """
        params = self._get_chat_params(stop=stop)
        should_stream = stream if stream is not None else self.streaming
        if should_stream:
            if len(prompts) > 1:
                raise ValueError(
                    f"WatsonxLLM currently only supports single prompt, got {prompts}"
                )
            generation = GenerationChunk(text="")
            stream_iter = self._stream(
                prompts[0], stop=stop, run_manager=run_manager, **kwargs
            )
            for chunk in stream_iter:
                if generation is None:
                    generation = chunk
                else:
                    generation += chunk
            assert generation is not None
            if isinstance(generation.generation_info, dict):
                llm_output = generation.generation_info.pop("llm_output")
                return LLMResult(generations=[[generation]], llm_output=llm_output)
            return LLMResult(generations=[[generation]])
        else:
            # Apply tools before generation
            if self.bound_tools:
                tool_output = self._evaluate_tools(self.bound_tools, prompts[0])
                print("Tool output: %s", tool_output)
                logger.info("Tool output: %s", tool_output)
                # You can include the tool output in the prompt or handle it as needed
                # For simplicity, we concatenate it to the prompt here
                #prompts[0] += "\n\n" + tool_output


                # Create a system prompt using the tool output
                system_prompt = (
                    f"You are an assistant with access to web search results. "
                    f"Provide a detailed answer to the user's query.\n\n"
                    f"User Query: {prompts[0]}\n\n"
                    f"Using the information below:\n"
                    f"Web Search Results: {tool_output}\n\n"
                )

                # Set the new prompt with the system prompt first
                prompts[0] = system_prompt
                    
  

            print("prompts",prompts)

            response = self.watsonx_model.generate(
                prompt=prompts, params=params, **kwargs
            )
            return self._create_llm_result(response)

    def _evaluate_tools(self, tool_instances: List[BaseTool], input_text: str) -> str:
        """Evaluate the provided tools and return their combined output.
        Args:
            tool_instances: List of tools to evaluate.
            input_text: Input text to provide to the tools.
        Returns:
            Combined output of the tool evaluations.
        """
        combined_output = []
        for tool in tool_instances:
            result = tool.invoke(input_text)
            #print("WebSearch Results:",result)
            #content = "\n\n".join([f"Content: {item['content']}" for item in result])
            content= "WebSearch Results: " + " ".join(result['content'] for result in result)
            combined_output.append(content)
        return "\n\n".join(combined_output)

    @classmethod
    def bind_tools(
        cls,
        tools: Sequence[Union[Dict[str, Any], Type[BaseModel], Callable, BaseTool]],
        *,
        tool_choice: Optional[Union[Dict[str, str], Literal["any", "auto"], str]] = None,
        **kwargs: Any,
    ) -> 'WatsonxLLM':
        """Bind tool-like objects to this chat model.
        Assumes model is compatible with OpenAI tool-calling API.
        Args:
            tools: A list of tool definitions to bind to this chat model.
                Can be a dictionary, pydantic model, callable, or BaseTool. Pydantic
                models, callables, and BaseTools will be automatically converted to
                their schema dictionary representation.
            tool_choice: Which tool to require to call.
                Options are:
                    name of the tool (str): calls corresponding tool;
                    "auto" or None: automatically selects a tool (including no tool);
                    "any": force at least one tool to be called;
                    or a dict of the form:
                        {"type": "tool", "name": "tool_name"},
                        or {"type": "any"},
                        or {"type": "auto"};
            **kwargs: Any additional parameters to pass to the
                :class:`~langchain.runnable.Runnable` constructor.
        """
        formatted_tools = [convert_to_openai_tool(tool)["function"] for tool in tools]
        instance = cls(**kwargs)
        instance.bound_tools = tools
        if tool_choice is not None:
            kwargs["tool_choice"] = tool_choice
        return instance

    def _create_llm_result(self, response: List[dict]) -> LLMResult:
        """Create the LLMResult from the choices and prompts."""
        generations = []
        for res in response:
            results = res.get("results")
            if results:
                finish_reason = results[0].get("stop_reason")
                gen = Generation(
                    text=results[0].get("generated_text"),
                    generation_info={"finish_reason": finish_reason},
                )
                generations.append([gen])
        final_token_usage = self._extract_token_usage(response)
        llm_output = {
            "token_usage": final_token_usage,
            "model_id": self.model_id,
            "deployment_id": self.deployment_id,
        }
        return LLMResult(generations=generations, llm_output=llm_output)

In [2]:
from dotenv import load_dotenv
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods

In [3]:
import getpass
import os
load_dotenv()  # Load environment variables from .env file
def _set_env(var: str):
    load_dotenv()  # Load environment variables from .env file
    env_var = os.getenv(var)
    if not env_var:
        env_var = getpass.getpass(f"{var}: ")
        os.environ[var] = env_var
    return env_var

_set_env("TAVILY_API_KEY")
api_key = _set_env("WATSONX_API_KEY")
project_id = _set_env("PROJECT_ID")
url = "https://us-south.ml.cloud.ibm.com"

In [4]:
   
# Create an instance of WatsonxLLM
# WatsonxLLM initialization
parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.SAMPLE.value,
    GenParams.MAX_NEW_TOKENS: 1000,
    GenParams.MIN_NEW_TOKENS: 50,
    GenParams.TEMPERATURE: 0.7,
    GenParams.TOP_K: 50,
    GenParams.TOP_P: 1
}
model_id = "ibm/granite-13b-instruct-v2"
watsonx_instance  = WatsonxLLM(
    model_id=model_id,
    url=url,
    apikey=api_key,
    project_id=project_id,
    params=parameters
)

In [5]:
watsonx_instance.invoke("How is the weather in Genova")

prompts ['How is the weather in Genova']


'Genoa has a moderate climate with hot summers and mild winters. \n\n\nThe average temperature in Genoa in January is 7.2°C, while in July it is 25.7°C. The rainiest months are October and November, while the sunniest are July and August.'

In [6]:
# Example usage:
tool = TavilySearchResults(max_results=4)

In [7]:
llm_with_tools = watsonx_instance.bind_tools(tools=[tool],model_id=model_id,url=url,apikey=api_key,project_id=project_id,params=parameters)   

In [8]:
response = llm_with_tools._generate(prompts=["Who is Ruslan Magana?"])
print(response)

Tool output: %s WebSearch Results: Ruslan Magana Vsevolodovna. National Institute for Nuclear Physics. Verified email at ge.infn.it - Homepage. Nuclear Physics Machine Learning Data Science Cloud Computing Big Data. ... R Magana, H Zheng, A Bonasera. International Journal of Modern Physics E 21 (01), 1250006, 2012. 4: 2012: I'm Ruslan Magana Vsevolodovna. I'm a Data Scientist, a Cloud Architect and a Physicist. About me. I am Data Scientist specializing in Artificial Intelligence, with a distinct focus on Neural Networks. My core expertise lies in Generative AI and prompt engineering. I possess a strong commitment to precision and boast an extensive track ... Nov 2020. Ruslan Magaña Vsevolodovna. The structure of odd-A Rh115,117 and Pd115,117 isotopes is studied by means of the neutron-proton interacting boson-fermion model (IBFM-2). JP=12+ quantum ... I am Data Scientist and Data Engineer. I have a Ph.D. in Physics and I am AWS certified in Machine Learning and Data Analytics - ruslan

In [9]:
response = llm_with_tools.invoke("Who is Ruslan Magana?")
print(response)

Tool output: %s WebSearch Results: I am Data Scientist and Data Engineer. I have a Ph.D. in Physics and I am AWS certified in Machine Learning and Data Analytics - ruslanmv Ruslan Magana Vsevolodovna. National Institute for Nuclear Physics. Verified email at ge.infn.it - Homepage. Nuclear Physics Machine Learning Data Science Cloud Computing Big Data. ... R Magana, H Zheng, A Bonasera. International Journal of Modern Physics E 21 (01), 1250006, 2012. 4: 2012: Nov 2020. Ruslan Magaña Vsevolodovna. The structure of odd-A Rh115,117 and Pd115,117 isotopes is studied by means of the neutron-proton interacting boson-fermion model (IBFM-2). JP=12+ quantum ... I'm Ruslan Magana Vsevolodovna. I'm a Data Scientist, a Cloud Architect and a Physicist. About me. I am Data Scientist specializing in Artificial Intelligence, with a distinct focus on Neural Networks. My core expertise lies in Generative AI and prompt engineering. I possess a strong commitment to precision and boast an extensive track .